#### LeetCode - SQL - #585

Refer [585. Investments in 2016](https://leetcode.com/problems/investments-in-2016/description/)

Write a solution to report the sum of all total investment values in 2016  `tiv_2016`, for all policyholders who:

-   have the same  `tiv_2015`  value as one or more other policyholders, and
-   are not located in the same city as any other policyholder (i.e., the (`lat, lon`) attribute pairs must be unique).

Round  `tiv_2016`  to  **two decimal places**.

In [ ]:
data_insurance = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
columns_insurance = ['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']
schema_insurance = {'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'}

In [ ]:
import pandas

pandas_dataframe_insurance = pandas.DataFrame(data=data_insurance, columns=columns_insurance).astype(schema_insurance)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
dataframe_insurance = spark_context.createDataFrame(pandas_dataframe_insurance)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec_tiv_2015 = Window.partitionBy(dataframe_insurance.tiv_2015)
window_spec_lat_lan = Window.partitionBy(dataframe_insurance.lat, dataframe_insurance.lon)

dataframe_insurance \
    = dataframe_insurance \
        .withColumn("tiv_2015_count", F.count("*").over(window_spec_tiv_2015)) \
        .withColumn("lat_lan_count", F.count("*").over(window_spec_lat_lan))
dataframe_insurance.show()

In [11]:
dataframe_insurance \
    .filter((dataframe_insurance.tiv_2015_count > 1) & (dataframe_insurance.lat_lan_count == 1)) \
    .select(F.round(F.sum(dataframe_insurance.tiv_2016), 2).alias("tiv_2016")) \
    .show()

+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+
